In [13]:
from cashe import get_train_test_split, get_testing_inputs, drop_nomodel_columns, get_training_inputs
from src.data_local import MergedDataPreprocessing, read_last_date, append_last_line
from src.utilities_rejection import DataFrameRejection
from src.lstm_encoder import LSTMEmbedding
from src.model_app import ModelApplicationDeployment
import numpy as np
from src.utilities_rejection import RejectionReasonLabeling
from src.model_app import ModelApplication

df_train, df_test = get_train_test_split('data/HJH/13-06-2024')

'''
if 'OUTCOME' not in df_test.columns:
    df_test['OUTCOME'] = ''
if 'SUBMIT_CLAIM_MESSAGE' not in df_test.columns:
    df_test['SUBMIT_CLAIM_MESSAGE'] = ''
'''

train_rej = RejectionReasonLabeling(df_train)
train_rej.recoginze_label()

train_rej = RejectionReasonLabeling(df_test)
train_rej.recoginze_label()

df_train['NPHIES_LABEL'] = np.where(df_train['NPHIES_CODE'] == 'MN-1-1', 1, 0)
df_test['NPHIES_LABEL'] = np.where(df_test['NPHIES_CODE'] == 'MN-1-1', 1, 0)

In [14]:
X_train, y_train, X_test, y_test = get_training_inputs(df_train,df_test)
X_train = drop_nomodel_columns(X_train)
X_test  = drop_nomodel_columns(X_test)

y_train['NPHIES_LABEL'] = df_train.NPHIES_LABEL; y_test['NPHIES_LABEL'] = df_test.NPHIES_LABEL

C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_8952\1485650554.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train['NPHIES_LABEL'] = df_train.NPHIES_LABEL; y_test['NPHIES_LABEL'] = df_test.NPHIES_LABEL


In [15]:
preprocessing_train = MergedDataPreprocessing(X_train)
preprocessing_test = MergedDataPreprocessing(X_test)

X_train_prep = preprocessing_train.columns_prep(service_encoding=False)
X_test_prep = preprocessing_test.columns_prep(service_encoding=False)

E:\Projects\Claims_Rejection\src\data_local.py:276: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column]= df[column].replace(column_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:284: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['PatientAgeRange']= df['PatientAgeRange'].replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:276: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(cop

In [16]:
lstm_embedding = LSTMEmbedding()

X_train_encoded = preprocessing_train.column_embedding(X_train_prep, is_service=True)
X_test_encoded  = preprocessing_train.column_embedding(X_test_prep, is_service=True)
X_train_encoded = preprocessing_train.column_embedding(X_train_encoded,is_service=False)
X_test_encoded  = preprocessing_train.column_embedding(X_test_encoded,is_service=False)

In [14]:
model_rejection = ModelApplication(X_train=X_train_encoded, y_train=y_train['OUTCOME'], X_test=X_test_encoded, y_test=y_test['OUTCOME'])

model_rejection.get_model(force_retrain=False,model_type='outcome')
y_rej_pred = model_rejection.model_predict(X_test)

Training is done


In [21]:
model_rejection = ModelApplication(X_train=X_train_encoded, y_train=y_train['NPHIES_LABEL'],
                                   X_test=X_test_encoded, y_test=y_test['NPHIES_LABEL'])

model_rejection.get_model(force_retrain=False,model_type='rejection')
y_reason_pred = model_rejection.model_predict(X_test)

XGBoost Model is loading from disk..


In [29]:
## check in rejection with reason
for i in range(len(y_rej_pred)):
    if y_rej_pred[i] == 1:
        y_reason_pred[i] = 0

df_test['Claim_Acceptance'] = y_rej_pred
df_test['Rejection_Reason'] =y_reason_pred

In [32]:
df_test.to_csv('predictions.csv',index=False)